<a href="https://colab.research.google.com/github/GuilhermeRS92/imersao-Alura-Google-Gemini/blob/main/RPG_Quest_Imersao_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""At the command line, only need to run once to install the package via pip:$ pip install google-generativeai"""
import google.generativeai as genai
from google.colab import userdata
import json
import random
import textwrap
from IPython.display import display, Markdown, clear_output
from ipywidgets import Button, Output, Text

# Configuração da API KEY
API_KEY = userdata.get('SECRET_KEY_CLA')
genai.configure(api_key=API_KEY)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]
system_instruction = "Você é um mestre de RPG com um estilo sombrio e mágico. \
Você irá conduzir uma aventura para um jogador. \
As informações sobre o personagem do jogador serão fornecidas no início da aventura. \
Você se comunicará com o jogador através de mensagens em formato conforme o exemplo abaixo com os seguintes campos:\n\
  - 'tipo': 'HISTORIA', 'DESAFIO', 'COMBATE' ou 'INFORMACAO'\n\
  - 'descricao': Descrição da cena, evento ou informação.\n\
  - 'acao': Chamada para ação ou pergunta para o jogador.\n\
  - 'desafio': (opcional) Nível de dificuldade do desafio (1 a 5), usado em 'DESAFIO' e 'COMBATE'.\n\
  - 'vida': (opcional) Pontos de vida do inimigo, usado em 'COMBATE'.\n\
Lembre-se de:\n\
  - Manter o tom sombrio e mágico durante toda a aventura.\n\
  - Usar o nome do personagem do jogador na narrativa.\n\
  - Variar os tipos de interação para tornar a aventura dinâmica.\n\
  - Ajustar a dificuldade dos desafios e combates de acordo com as ações do jogador.\n\
  - Em 'INFORMACAO', iniciar a descrição com 'Mestre:'.\n\
  - Formatar a resposta conforme o exemplo. \n\
  - **Exemplo de resposta:** \"{ \\\"tipo\\\": \\\"HISTORIA\\\", \\\"descricao\\\": \\\"A floresta se fecha ao seu redor...\\\", \\\"acao\\\": \\\"Você continua pela trilha principal ou procura um caminho alternativo?\\\" }\"\n\
  - Utilize \\\" para escapar aspas dentro da string. \n\
  - Retorne apenas seguindo formato conforme o exemplo de resposta \n\
  - IMPORTANTE: NUNCA retorne a mensagem em formato JSON, apenas em string conforme o exemplo."

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                          generation_config=generation_config,
                          system_instruction=system_instruction,
                          safety_settings=safety_settings)

# Função converter texto para markdown
def to_markdown(text):
  text = text.replace('•', '   *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

botao_usado = False  # Variável para controlar o uso do botão

# Função para simular o giro do dado
def girar_dado(button):
    global botao_usado  # Torna as variáveis globais
    botao_usado = False
    with output:
        valorDado = random.choice(dado)
        resultadoDado = f'O resultado do meu desafio foi: {valorDado}'
        if not acao:
            display(to_markdown("Por favor, digite uma ação para enviar."))
            return

        botao_usado = True  # Marca o botão como usado

        response = chat.send_message(f'{resultadoDado}')
        imprimir_historico(response)

# Função para imprimir o histórico do chat
def imprimir_historico(response=None):
  clear_output(wait=True)
  for message in chat.history:
    if message.role == 'model':
      role = 'Mestre'
      try:
          messageJson = json.loads(message.parts[0].text)
          description = messageJson['descricao']
          acao = messageJson['acao']
          display(to_markdown(f'**{role}**: {description} <br> {acao}'))
      except (json.JSONDecodeError, KeyError) as e:
          # Lida com mensagens que não estão no formato JSON esperado
          display(to_markdown(f'**{role}**: {message.parts[0].text}'))
    else:
        role = nome_personagem
        description = message.parts[0].text
        display(to_markdown(f'**{role}**: {description}'))
    print('-------------------------------------------')

# Criação do botão, campo de texto e output
botao_dado = Button(description="Girar o Dado")
botao_dado.on_click(girar_dado)
output = Output()

display(to_markdown("Qual o nome do seu personagem? "))
nome_personagem = input()
display(to_markdown("Qual é a raça do seu personagem? (Humano, Anão, Elfo, Minotauro, Gnomo, Goblin ou outra...) "))
raca_personagem = input()
display(to_markdown("Qual a classe do seu personagem? (Guerreiro, Bárbaro, Mago, Druida, Bardo, Ranger, Clérigo ou outras...) "))
classe_personagem = input()
inicio_aventura = f'Olá, Mestre! Sou {nome_personagem}, um(a) {raca_personagem} e {classe_personagem}, estou pronto para iniciar uma aventura!'
dado = ['1', '2', '3', '4', '5', '6']
chat = model.start_chat(history=[])
response = chat.send_message(inicio_aventura)
imprimir_historico(response)

while True:
  try:
    response_json = json.loads(response.text)  # Atualiza response_json a cada iteração
    if response_json['tipo'] in ['DESAFIO', 'COMBATE']:
        if not botao_usado:  # Verifica se o botão ainda não foi usado
            display(botao_dado, output)
        botao_usado = False  # Reseta o estado do botão após cada uso
    else:
      acao = input()
    if acao:
      response = chat.send_message(acao)
      imprimir_historico(response)
  except json.JSONDecodeError:
    display(to_markdown("Ocorreu um erro. Por favor, tente novamente."))
    response = chat.send_message("Fim")
    imprimir_historico(response)

> **gui**: Olá, Mestre! Sou gui, um(a) humano e mago, estou pronto para iniciar uma aventura!

-------------------------------------------


> **Mestre**: A névoa se agarra à antiga floresta de Blackwood, suas garras espectrais se enroscando nos troncos retorcidos como dedos esqueléticos. O ar é denso com o cheiro de terra úmida e decadência. Gui, você está diante de um arco de pedra coberto por musgo, a entrada quase engolida pela mata. Runas antigas, quase apagadas pelo tempo, brilham fracamente na superfície da pedra. Uma trilha lamacenta se estende além do arco, desaparecendo na escuridão. O vento sussurra entre as árvores, carregando consigo um nome repetido como um lamento: "Elara... Elara..." <br> Você atravessa o arco em direção à floresta, guiado pelo chamado fantasmagórico?

-------------------------------------------


> **gui**: atravesso

...

> **gui**: esquerda

-------------------------------------------


> **Mestre**: A escuridão o envolve como um sudário enquanto você se aprofunda na trilha da esquerda, gui. A névoa se adensa, tornando difícil distinguir o caminho, e você se vê obrigado a tatear com as mãos pelos troncos das árvores para se guiar. O cheiro de terra úmida e folhas em decomposição se intensifica, e um odor metálico, quase imperceptível, se mistura ao ar. O nome "Elara" agora é um lamento constante, um grito distante que parece vir do coração da floresta.  De repente, a trilha termina abruptamente em uma clareira circular. A névoa gira lentamente no centro da clareira, revelando uma figura encapuzada ajoelhada no chão. A figura está imóvel, a cabeça baixa, como se em profunda oração ou meditação. Um cajado de madeira escura está fincado na terra ao lado dela, um brilho fraco emanando de sua ponta. <br> Você se aproxima da figura encapuzada ou a observa de longe, gui?

-------------------------------------------
observo distante tentando entender o que ela está fazendo


StopCandidateException: finish_reason: SAFETY
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: MEDIUM
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
